In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[119]: DataFrame[key: string, value: string]

In [0]:
dbutils.fs.ls("/mnt/blobed/")
calendario = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/calendario.csv")
feriados = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/feriados_nacionais.csv")

In [0]:
#Calendario - Remover Hora da Coluna Data
#Feriado - Calcular dias e preencher/ Criar Coluna de Dia útil 0/1 e Tipo (Dia útil | fim de semana | nome do feriado)

calendario = calendario.withColumn("Data", F.to_date(F.to_timestamp(col("Data"), "dd/MM/yyyy")))
feriados = feriados.na.drop(how = "any")

feriados = feriados.withColumn("Data", F.to_date(F.to_timestamp(col("Data"), "dd/MM/yyyy")))

In [0]:
#Adicionado ID para obtenção de coluna ID
calendario = calendario.select("*").withColumn("id", monotonically_increasing_id())

def diaDaSemanaNumero(colunaNum):
    if colunaNum % 7 == 3:
        return "7"
    if colunaNum % 7 == 4:
        return "1"
    if colunaNum % 7 == 5:
        return "2"
    if colunaNum % 7 == 6:
        return "3"
    if colunaNum % 7 == 0:
        return "4"
    if colunaNum % 7 == 1:
        return "5"
    if colunaNum % 7 == 2:
        return "6"
    
def diaUtil(ColunaSemanaNulls, ColunaTipo):
    if (ColunaSemanaNulls == "6" or ColunaSemanaNulls ==  "7") and ColunaTipo == None:
        return "Fim de Semana"
    elif ColunaTipo == None:
        return "Útil"
    else:
        return ColunaTipo
    
def numeroDia(ColunaUtil):    
    if ColunaUtil == "Útil":
        return 1
    else:
        return 0

    
diasNum = udf(diaDaSemanaNumero, StringType())
diaTeste = udf(diaUtil, StringType())
diasBinario = udf(numeroDia, StringType())

#Lembrar que 1 é segunda-feira e 7 é domingo
calendario = calendario.withColumn("Número", diasNum('id'))


#Left Join do Calendario com Feriados
calendario = calendario.join(feriados,'Data','left')

#Drop do Dia da Semana
calendario = calendario.drop("Dia da Semana")

#Renome da coluna
calendario = calendario.withColumnRenamed("Feriado","Tipo")

#Adição de dias que não são feriados
calendario = calendario.withColumn("Tipo", diaTeste('Número','Tipo'))

#Adição do binário
calendario = calendario.withColumn("0-1", diasBinario('Tipo'))

#Retirada da coluna id
calendario = calendario.drop("id")

display(calendario)


Data,Número,Tipo,0-1
2016-06-30,4,Útil,1
2016-07-01,5,Útil,1
2016-07-02,6,Fim de Semana,0
2016-07-03,7,Fim de Semana,0
2016-07-04,1,Útil,1
2016-07-05,2,Útil,1
2016-07-06,3,Útil,1
2016-07-07,4,Útil,1
2016-07-08,5,Útil,1
2016-07-09,6,Fim de Semana,0
